In [3]:
import os
import pandas as pd
import psycopg2 as pg
from sshtunnel import SSHTunnelForwarder
from dotenv import load_dotenv

# Check for .env file and if found then load it
if os.path.exists(".env"):
    # Load environment variables from .env
    load_dotenv()
else:
    # Create .env file from .env.example
    with open(".env.example", "r") as f:
        example = f.read()
    with open(".env", "w") as f:
        f.write(example)
    print("Created .env file! Now paste into .env values from password manager")

In [4]:
# Ref: https://towardsdatascience.com/how-to-connect-to-a-postgresql-database-with-python-using-ssh-tunnelling-d803282f71e7
def connect_ppg2():
    try:
        print("Connecting to the PostgreSQL Database...")

        ssh_tunnel = SSHTunnelForwarder(
            (os.environ["SSH_HOST"]),
            ssh_username=os.environ["SSH_USERNAME"],
            ssh_private_key=os.environ["SSH_KEY_PATH"],
            ssh_private_key_password=os.environ["SSH_KEY_PASSWORD"],
            remote_bind_address=(os.environ["PGHOST"], 5432),
        )
        ssh_tunnel.start()

        conn = pg.connect(
            host="localhost",
            port=ssh_tunnel.local_bind_port,
            user=os.environ["PGUSER"],
            password=os.environ["PGPASSWORD"],
            database=os.environ["PGDATABASE"],
        )
    except:
        print("Connection Has Failed...")

    return conn


def create_df_from_ppg2(query, conn):
    print("Executing SQL Query & Saving To DataFrame...")
    return pd.read_sql_query(query, conn)

In [ ]:
query = """
select
  id,
  name
from
  accounts
order by
  created_at desc
limit 
  10
"""

df = create_df_from_ppg2(query, connect_ppg2())
df.head()

## Clean up

In [5]:
# Delete .env file
os.remove(".env")